In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pandas as pd
import numpy as np
import time

print("Torch version: ", torch.__version__)
print("Is CUDA available?", torch.cuda.is_available())
print("Torch cuda version: ", torch.version.cuda if torch.cuda.is_available() else "None")
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")

In [ ]:
# Załadowanie danych
data = pd.read_csv("../data/ObesityDataSet.csv")

# Wyodrębnij cechy i etykiety
X = data.drop("NObeyesdad", axis=1)
y = data["NObeyesdad"]
# One-hot encoding dla cech kategorycznych
X = pd.get_dummies(X)

# Label encoding dla etykiet klasowych
le = LabelEncoder()
y_encoded = le.fit_transform(y)
# Skalowanie danych
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# Podział danych
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.linear(x)  # logits
class NeuralNetworkModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(NeuralNetworkModel, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, output_dim)
        )

    def forward(self, x):
        return self.net(x)


In [ ]:
def train_model(device):
    # model = LogisticRegressionModel(X_train_tensor.shape[1], len(np.unique(y_encoded)))
    model = NeuralNetworkModel(X_train_tensor.shape[1], len(np.unique(y_encoded)))
    model.to(device)

    # Funkcja straty dla klasyfikacji wieloklasowej.
    criterion = nn.CrossEntropyLoss()
    # Optymalizator
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01) #SGD / Adam

    start = time.time()
    for epoch in range(20):
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
    end = time.time()

    return model, end - start
device_cpu = torch.device("cpu")
device_gpu = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Trening na CPU...")
model_cpu, time_cpu = train_model(device_cpu)
print(f"Czas treningu na CPU: {time_cpu:.4f} sek.")

if torch.cuda.is_available():
    print("\nTrening na GPU...")
    model_gpu, time_gpu = train_model(device_gpu)
    print(f"Czas treningu na GPU: {time_gpu:.4f} sek.")
else:
    print("GPU niedostępne.")

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import classification_report, accuracy_score

model_cpu.eval()  # tryb ewaluacji

with torch.no_grad():  # nie śledzimy gradientów w ewaluacji
    outputs_test = model_cpu(X_test_tensor.to(device_cpu)) # device_gpu
    predicted_classes = torch.argmax(outputs_test, dim=1).cpu().numpy()

# Ewaluacja
print("=== Ewaluacja na zbiorze testowym CPU ===")
print("Accuracy:", accuracy_score(y_test, predicted_classes))
print("\nClassification Report:")
print(classification_report(y_test, predicted_classes, target_names=le.classes_))

if torch.cuda.is_available():
    model_gpu.eval()  # tryb ewaluacji

    with torch.no_grad():  # nie śledzimy gradientów w ewaluacji
        outputs_test = model_gpu(X_test_tensor.to(device_gpu)) # device_gpu
        predicted_classes = torch.argmax(outputs_test, dim=1).cpu().numpy()

    # Ewaluacja
    print("=== Ewaluacja na zbiorze testowym GPU ===")
    print("Accuracy:", accuracy_score(y_test, predicted_classes))
    print("\nClassification Report:")
    print(classification_report(y_test, predicted_classes, target_names=le.classes_))